In [4]:
import geopandas as gpd

wards = gpd.read_file("data/cpt_boundary_wards.gpkg")
districts = gpd.read_file("data/ODP/Development_Management_Districts/SL_PBDM_RGN.shp")

In [3]:
streets = gpd.read_file("data/cpt_network_cleaned.gpkg")

In [8]:
wards.head(2)

,OBJECTID,WARD_NAME,WARD_YEAR,geometry
0,538,1,2021,"MULTIPOLYGON (((2070517.326 -4013255.623, 2070..."
1,539,2,2021,"MULTIPOLYGON (((2070958.885 -4013973.53, 20710..."


In [9]:
streets.head(2)

,u,v,geometry
0,5473988567113339372,7519887590434988504,"LINESTRING (18.35125 -33.98829, 18.35389 -33.9..."
1,-1658900339951337858,7519887590434988504,"LINESTRING (18.37143 -33.97054, 18.37176 -33.9..."


In [10]:
districts.head(2)

,OBJECTID,PBDM_RGN_N,SHAPE_Leng,SHAPE_Area,geometry
0,809,CAPE FLATS,72440.154512,1.953103e+08,"POLYGON ((2060444.866 -4022051.414, 2060594.81..."
1,810,MITCHELLS PLAIN/KHAYELITSHA,77410.762312,2.290795e+08,"POLYGON ((2084002.546 -4022865.772, 2084226.73..."


In [11]:
wards = wards.to_crs("EPSG:3857")
streets = streets.to_crs("EPSG:3857")
districts = districts.to_crs("EPSG:3857")

In [12]:
# Check for invalid or empty geometries in 'streets'
invalid_streets = streets[~streets.is_valid | streets.is_empty]
print(f"Invalid or empty geometries: {len(invalid_streets)}")

Invalid or empty geometries: 0


In [13]:
# Step 1: Calculate street segment lengths
streets['length_m'] = streets.geometry.length

# Step 2: Spatial join with wards
streets_in_wards = gpd.sjoin(streets, wards[['WARD_NAME', 'geometry']], how='inner', predicate='intersects')

# Step 3: Group by ward and sum lengths
ward_lengths = streets_in_wards.groupby('WARD_NAME')['length_m'].sum().reset_index()

# Step 4: Merge back with ward geometries
wards_with_length = wards[['WARD_NAME', 'geometry']].merge(ward_lengths, on='WARD_NAME', how='left')
wards_with_length['length_m'] = wards_with_length['length_m'].fillna(0)

In [14]:
# Spatial join with districts
streets_in_districts = gpd.sjoin(streets, districts[['PBDM_RGN_N', 'geometry']], how='inner', predicate='intersects')

# Group by district and sum lengths
district_lengths = streets_in_districts.groupby('PBDM_RGN_N')['length_m'].sum().reset_index()

# Merge back with district geometries
districts_with_length = districts[['PBDM_RGN_N', 'geometry']].merge(district_lengths, on='PBDM_RGN_N', how='left')
districts_with_length['length_m'] = districts_with_length['length_m'].fillna(0)

In [16]:
# For wards
wards_with_length['area_km2'] = wards_with_length.geometry.area / 1e6
wards_with_length['length_per_km2'] = wards_with_length['length_m'] / wards_with_length['area_km2']

# For districts
districts_with_length['area_km2'] = districts_with_length.geometry.area / 1e6
districts_with_length['length_per_km2'] = districts_with_length['length_m'] / districts_with_length['area_km2']

In [17]:
# Save wards with length to GeoPackage
wards_with_length.to_file("wards_street_length.gpkg", layer="wards", driver="GPKG")

# Save districts with length to GeoPackage
districts_with_length.to_file("districts_street_length.gpkg", layer="districts", driver="GPKG")